In [54]:
from sklearn.cluster import AffinityPropagation
import os
import re
import sys
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from gensim import corpora, models
from nltk.corpus import wordnet
from nltk import word_tokenize, pos_tag
import traceback
import numpy as np
import matplotlib.pyplot as plt
import string
import nltk
import nltk.data
from itertools import combinations
%matplotlib notebook

In [55]:
corpus_path = "/Users/zhouyou/Downloads/bbc news dataset"
all_path = "/Users/zhouyou/Downloads/summ/"

In [56]:
# load dataset
file_dirs = os.listdir(corpus_path)
# news dataset dir
corpus_text = []
for file_dir in file_dirs:
    if os.path.isdir(corpus_path + '/' + file_dir):
        # different category dir
        files = os.listdir(corpus_path + '/' + file_dir)
        count = 0
        for file in files:
            try:
                # 每种取了200篇
                if count < 200:
                    # txt file
                    f = open(corpus_path + '/' + file_dir + "/" + file)
                    text_content = ""
                    next(f)
                    for line in f.readlines():
                        if line != "\n":
                            text_content = text_content + line
                    text_content = text_content.replace("\n", "")
                    count = count + 1
                else:
                    break
                corpus_text.append(text_content)
            except:
                pass

In [57]:
# tokenize
tokenizer = RegexpTokenizer(r'\w+')

en_stop = get_stop_words('en')

p_stemmer = WordNetLemmatizer()

tokennized_text = []
tokennized_word = []

for i in corpus_text:
    tokens = tokenizer.tokenize(i)
    stopped_tokens = [i.lower() for i in tokens if not i in en_stop]
    # stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    stemmed_tokens = [p_stemmer.lemmatize(i) for i in stopped_tokens]
    final_tokens = []
    for i in stemmed_tokens:
        if len(i) > 1:
            final_tokens.append(i)
    # print(stemmed_tokens)
    tokennized_word.extend(final_tokens)
    tokennized_text.append(final_tokens)




In [58]:
# 计算词频，删除频率太低的词
corpus_word_frequency = {}
for i in tokennized_word:
    if i in corpus_word_frequency:
        corpus_word_frequency[i] = corpus_word_frequency[i] + 1
    else:
        corpus_word_frequency.setdefault(i, 1)

In [59]:
low_frequency_word = []
for i in corpus_word_frequency.keys():
    if corpus_word_frequency[i] <= 2:
        low_frequency_word.append(i)

In [60]:
for i in tokennized_text:
    for j in low_frequency_word:
        if j in i:
            i.remove(j)

In [61]:
rebuild_text = []
for i in tokennized_text:
    temp = ""
    for word in i:
        temp = temp + " " + word
    rebuild_text.append(temp)

In [65]:
# 读文本
def readfile(path):
    # 遍历文件夹
    files = os.listdir(path)
    files.sort()
    for file in files:
        raw_text = ''
        raw_text_list = []
        if not os.path.isdir(file):
            # print("File name: " + file)
            f = open(path + "/" + file)
            for line in f.readlines():
                searchObj = re.findall(r'SNO=(.*?)>(.*?)</S>', line)
                if searchObj:
                    # print(searchObj)
                    # print(searchObj[0][0])
                    if searchObj[0][0] == "\"1\"":
                        raw_text = raw_text + searchObj[0][1] + '.' + ' '
                        raw_text_list.append(searchObj[0][1])
                    else:
                        raw_text = raw_text + searchObj[0][1] + ' '
                        raw_text_list.append(searchObj[0][1])
                    # print(s)
            yield file, raw_text, raw_text_list

In [66]:
def getDoc(document_path):
    per_raw_text = readfile(document_path)

    all_raw_text = []
    all_filenames = []
    all_raw_text_list = []

    for filename, raw_text, raw_text_list in per_raw_text:
        all_filenames.append(filename)
        all_raw_text.append(raw_text)
        all_raw_text_list.append(raw_text_list)

    tokenizer = RegexpTokenizer(r'\w+')
    
    sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

    en_stop = get_stop_words('en')

    p_stemmer = WordNetLemmatizer()


    doc = []
    sentences = sent_tokenizer.tokenize(all_raw_text[0])
    for sent in sentences:
        tokens = tokenizer.tokenize(sent)
        stopped_tokens = [i for i in tokens if not i in en_stop]
        stemmed_tokens = [p_stemmer.lemmatize(i) for i in stopped_tokens]
        doc.append(stemmed_tokens)
        
    return doc

In [67]:
doc = getDoc(all_path + "2/docsent")

In [68]:
from sklearn.metrics.pairwise import cosine_similarity
def cal_cos_sim(temp, document_topic_list, document_topic_porb):
    cos_sim = []
    count = 0
    judge_topic_list = []
    
    for j in temp:
        a, b = j
        judge_topic_list.append(a)
    union_set = set(document_topic_list).union(set(judge_topic_list))
    
    document_topic_porb_kv = {}
    for j in document_topic_porb:
        a, b = j
        document_topic_porb_kv.setdefault(a, b)
        
    judge_topic_list_kv = {}
    for j in temp:
        a, b = j
        judge_topic_list_kv.setdefault(a, b)
    
    document_topic_porb_cal_coss = []
    judge_topic_list_cal_coss = []
    for k in union_set:
        document_topic_porb_cal_coss.append(document_topic_porb_kv.get(k, 0))
        judge_topic_list_cal_coss.append(judge_topic_list_kv.get(k, 0))
        
        
    cos_sim.append(cosine_similarity(np.array([document_topic_porb_cal_coss]), np.array([judge_topic_list_cal_coss]))[0][0])
    return cos_sim
    

In [70]:
# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(tokennized_text)

# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in tokennized_text]
    
# generate LDA model
ldamodel = models.ldamodel.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=20)

# document_topic: 文本的分布
doc_input = []
for i in doc:
    doc_input = doc_input + i
doc_bow = dictionary.doc2bow(doc_input)
document_topic = ldamodel.get_document_topics(doc_bow)
        
# 计算相似度
document_topic_porb = document_topic
document_topic_list = []
for i in document_topic:
    a, b = i
    document_topic_list.append(a)

In [76]:
for i in range(2, 15):
    combs = list(combinations(range(0, 18), i))
    # 所有拍立组合
    bestCosSim = 0
    bestComb = None
    for comb in combs:
        # 某一个排列组合计算cos_sim
        randomCombSent = []
        for sentNum in comb:
            randomCombSent = randomCombSent + doc[sentNum]
        # 计算cos_sim后比较，大就存下来
        summ_bow = dictionary.doc2bow(randomCombSent)      
        res = ldamodel.get_document_topics(bow=summ_bow)
        cos_sim = cal_cos_sim(res, document_topic_list, document_topic_porb)
        if cos_sim[0] > bestCosSim:
            bestCosSim = cos_sim
            bestComb = comb
        
    print("i=" + str(i))
    select_sent = []
    for sent in bestComb:
        select_sent.append(sent + 1)
    print("best sentence=" + str(select_sent))
    

i=2
best sentence=[3, 17]
i=3
best sentence=[3, 14, 17]
i=4
best sentence=[3, 9, 12, 17]
i=5
best sentence=[2, 3, 9, 11, 17]
i=6
best sentence=[2, 3, 9, 14, 17, 18]
i=7
best sentence=[2, 3, 9, 11, 12, 13, 17]
i=8
best sentence=[2, 3, 6, 7, 9, 14, 17, 18]
i=9
best sentence=[2, 3, 6, 7, 9, 11, 12, 13, 17]
i=10
best sentence=[2, 3, 6, 7, 8, 9, 12, 13, 17, 18]
i=11
best sentence=[3, 6, 8, 9, 10, 12, 13, 14, 16, 17, 18]
i=12
best sentence=[4, 5, 6, 7, 8, 9, 11, 12, 14, 16, 17, 18]
i=13
best sentence=[1, 2, 3, 4, 6, 7, 9, 11, 12, 15, 16, 17, 18]
i=14
best sentence=[1, 2, 3, 4, 5, 7, 9, 10, 11, 12, 14, 16, 17, 18]
